In [61]:
import pandas as pd
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from pprint import pprint
from tqdm.auto import tqdm
from tqdm import tqdm_notebook
from scipy.sparse import hstack
tqdm.pandas()

In [2]:
df_description = pd.read_csv('./data/data_definition.txt', sep='\t')

df_train_genba = pd.read_csv('./data/train_genba.tsv', sep='\t')
df_train_goto = pd.read_csv('./data/train_goto.tsv', sep='\t')

df_train = df_train_goto.merge(df_train_genba, on='pj_no', how='left')
df_train.drop('id', axis=1, inplace=True)

df_test_genba = pd.read_csv('./data/test_genba.tsv', sep='\t')
df_test_goto = pd.read_csv('./data/test_goto.tsv', sep='\t')

df_test = df_test_goto.merge(df_test_genba, on='pj_no', how='left')
df_test.drop('id', axis=1, inplace=True)

## Preprocessing

In [3]:
def fill_city_name(name):
    if '市' not in name and '郡' not in name:
        name = '市' + name
    return name

def split_address(df):
    df['jukyo'] = df['jukyo'].str.slice(start=3).str.replace(r'[ヶｹ]', 'ケ')
    df['jukyo'] = df['jukyo'].apply(fill_city_name)
    city_split = df['jukyo'].str.split(r'[市郡]', n=1, expand=True)
    df['city'] = city_split[0]
    street_split = city_split[1].str.split(r'[町区]', n=1, expand=True)
    df['street'] = street_split[0]
    df['address_detail'] = street_split[1].str.strip().replace('', None)
    return df

df_train = split_address(df_train)
df_test = split_address(df_test)

df_train.drop(['kaoku_um', 'toshikuiki2', 'shu_sogi'], axis=1, inplace=True)
df_test.drop(['kaoku_um', 'toshikuiki2', 'shu_sogi'], axis=1, inplace=True)

In [4]:
def combine(row, combine_list, tup):
    l = set()
    for col in tup:
        if pd.notnull(row[col]):
            l.add(row[col])
    combine_list.append(','.join(l))


combine_cols = [('yoto', 100), ('road_hk', 100)]
for i, tup in enumerate([['yoto1', 'yoto2'], ['road1_hk', 'road2_hk', 'road3_hk', 'road4_hk']]):
    combine_train = []
    combine_test = []
    
    combine_col_name = combine_cols[i][0]
    _ = df_train.apply(lambda row: combine(row, combine_train, tup), axis=1)
    _ = df_test.apply(lambda row: combine(row, combine_test, tup), axis=1)

    count_vectorizer = CountVectorizer(min_df=combine_cols[i][1])
    combine_train_matrix = count_vectorizer.fit_transform(combine_train).todense()
    combine_test_matrix = count_vectorizer.transform(combine_test).todense()
    for i in range(combine_train_matrix.shape[1]):
        df_train['%s_%d' % (combine_col_name, i)] = combine_train_matrix[:, i]
        df_test['%s_%d' % (combine_col_name, i)] = combine_test_matrix[:, i]
    for col in tup:
        df_train.drop(col, axis=1, inplace=True)
        df_test.drop(col, axis=1, inplace=True)

## Train

In [5]:
continue_features = list(df_description[(df_description['データ型'] == '数値') & (df_description['項目名'] != 'pj_no')]['項目名'])
objective = 'keiyaku_pr'
categorical_features = list(df_train)

for f in continue_features+[objective]:
    if f in categorical_features:
        categorical_features.remove(f)

for col in categorical_features:
    dt = df_train[col].dtype
    if dt == int or dt == float:
        df_train[col].fillna(-1, inplace=True)
        df_test[col].fillna(-1, inplace=True)
    else:
        df_train[col].fillna('', inplace=True)
        df_test[col].fillna('', inplace=True)
        
for col in categorical_features:
    if col not in ['pj_no']:
        vc = dict(df_train[col].value_counts())
        for k, v in vc.items():
            if v <= 3:
                df_train[col].replace(k, 'Low_freq', inplace=True)
                df_test[col].replace(k, 'Low_freq', inplace=True)
        le = LabelEncoder()
        df_train[col] = le.fit_transform(df_train[col])
        mapping = {v: i for i, v in enumerate(le.classes_)}
        df_test[col] = df_test[col].apply(lambda x: mapping[x] if x in mapping else -1)
        
for col in continue_features:
    if col != 'keiyaku_pr':
        scaler = MinMaxScaler()
        df_train[col] = scaler.fit_transform(df_train[col].values.reshape(-1, 1))
        df_test[col] = scaler.transform(df_test[col].values.reshape(-1, 1))
    
df_test['keiyaku_pr'] = 0

In [6]:
df_train

,pj_no,keiyaku_pr,tc_mseki,tt_mseki,levelplan,fukuin,road_st,magutchi,setsudo_hi,setsudo_kj,...,yoto_2,yoto_3,yoto_4,yoto_5,yoto_6,yoto_7,road_hk_0,road_hk_1,road_hk_2,road_hk_3
0,0,39800000,0.131636,0.729005,8,0.159363,1,0.204969,24,3,...,0,0,0,0,0,0,0,0,0,0
1,1,22300000,0.175238,0.734101,8,0.167331,1,0.192547,7,2,...,0,0,1,0,0,0,0,1,0,0
2,2,19800000,0.095601,0.599930,8,0.159363,1,0.229814,24,2,...,0,0,1,0,0,0,0,0,0,0
3,3,33990000,0.220124,0.739825,8,0.159363,1,0.217391,13,2,...,0,0,0,0,0,0,0,0,0,0
4,4,30800000,0.135290,0.621361,8,0.159363,1,0.279503,2,3,...,0,0,1,0,0,0,1,0,0,1
5,4,35800000,0.135322,0.621361,8,0.159363,1,0.144928,13,3,...,0,0,1,0,0,0,1,0,0,1
6,4,35300000,0.135322,0.615567,5,0.159363,1,0.140787,13,3,...,0,0,1,0,0,0,1,0,0,1
7,5,21496297,0.140048,0.690750,8,0.159363,1,0.161491,24,2,...,0,0,0,1,0,0,1,1,0,0
8,6,19200000,0.399568,0.000000,8,0.282869,1,0.213251,14,3,...,0,0,0,0,0,0,0,0,1,1
9,6,17900000,0.346758,0.000000,8,0.294821,1,0.163561,1,3,...,0,0,0,0,0,0,0,0,1,1


In [83]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

#continue_features.remove('keiyaku_pr')
splitter = KFold(n_splits=5, shuffle=True, random_state=28)
prediction_list = []
best_scores = []
df_train.fillna(0, inplace=True)
df_test.fillna(0, inplace=True)
for train_idx, valid_idx in splitter.split(df_train):
    train, valid = df_train.iloc[train_idx], df_train.iloc[valid_idx]
    test = df_test[df_train.drop(objective, axis=1).columns]
    encoder = OneHotEncoder(categories='auto', handle_unknown='ignore')
    train_cat_one_hot = encoder.fit_transform(train[categorical_features])
    valid_cat_one_hot = encoder.transform(valid[categorical_features])
    test_cat_one_hot = encoder.transform(test[categorical_features])
    X_train, y_train = hstack([train[continue_features].values, train_cat_one_hot]), np.log(train['keiyaku_pr']+1)
    X_valid, y_valid = hstack([valid[continue_features].values, valid_cat_one_hot]), np.log(valid['keiyaku_pr']+1)
    regressor = linear_model.Ridge(alpha=2)
    regressor.fit(X_train, y_train)
    pred_val = regressor.predict(X_valid)
    prediction_list.append(regressor.predict(hstack([test[continue_features].values, test_cat_one_hot])))
    best_scores.append(mean_squared_error(y_valid, pred_val))
    
print("5-fold cv mean l2 %.8f" % np.mean(best_scores))

df_submission = pd.read_csv('./data/sample_submit.tsv', sep='\t', names=['id', 'pred'])

df_submission['pred'] = np.exp(np.mean(prediction_list, axis=0))-1
df_submission.to_csv('submission_ridge.tsv', sep='\t', header=None, index=False)

5-fold cv mean l2 0.01409391


In [ ]:
splitter = KFold(n_splits=5, shuffle=True, random_state=28)
prediction_list = []
best_scores = []
for train_idx, valid_idx in splitter.split(df_train):
    train, valid = df_train.iloc[train_idx], df_train.iloc[valid_idx]
    X_train, y_train = train.drop('keiyaku_pr', axis=1), np.log(train['keiyaku_pr']+1)
    X_valid, y_valid = valid.drop('keiyaku_pr', axis=1), np.log(valid['keiyaku_pr']+1)
    regressor = lgb.LGBMRegressor(n_estimators=20000, learning_rate=0.01, silent=False, random_state=28,
                                 bagging_fraction=0.9, bagging_freq=1, feature_fraction=0.8, lambda_l1=0.01)
    regressor.fit(X_train, y_train, eval_set=(X_valid, y_valid), early_stopping_rounds=500,
                  categorical_feature=categorical_features)
    prediction_list.append(regressor.predict(df_test[df_train.drop(objective, axis=1).columns]))
    best_scores.append(regressor.best_score_['valid_0']['l2'])

print("5-fold cv mean l2 %.8f" % np.mean(best_scores))

df_submission = pd.read_csv('./data/sample_submit.tsv', sep='\t', names=['id', 'pred'])

df_submission['pred'] = np.exp(np.mean(prediction_list, axis=0))-1
df_submission.to_csv('submission.tsv', sep='\t', header=None, index=False)

# 0.01225944